In [0]:
from delta.tables import DeltaTable

stage_table_name = "incremental_load.default.orders_stage"
target_table_name = "incremental_load.default.orders_target"

# Read stage data
stage_df = spark.read.table(stage_table_name)

# Check if target table exists (use Spark catalog API, not _jsparkSession)
if not spark.catalog.tableExists(target_table_name):
    # Create target table with the same schema as stage
    stage_df.write.format("delta").saveAsTable(target_table_name)

else:
    # Load target table as DeltaTable
    target_table = DeltaTable.forName(spark, target_table_name)

    # Merge condition: match on tracking_num
    merge_condition = "stage.tracking_num = target.tracking_num"

    # Perform merge (delete old record, insert new one)
    target_table.alias("target") \
        .merge(stage_df.alias("stage"), merge_condition) \
        .whenMatchedDelete() \
        .whenNotMatchedInsertAll() \
        .execute()
